In [30]:
import os
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from tqdm import tqdm
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
import numpy as np
from collections import Counter
from sklearn.metrics.cluster import contingency_matrix

directory = r'C:\Users\DELL\Desktop\Ass3\Doc50'
directoryA = r'C:\Users\DELL\Desktop\Ass3\Doc50 GT\C1'
directoryB = r'C:\Users\DELL\Desktop\Ass3\Doc50 GT\C2'
directoryC = r'C:\Users\DELL\Desktop\Ass3\Doc50 GT\C3'
directoryD = r'C:\Users\DELL\Desktop\Ass3\Doc50 GT\C4'
directoryE = r'C:\Users\DELL\Desktop\Ass3\Doc50 GT\C5'
file_contents = []
name = []
remove = ""
C1 = []
C2 = []
C3 = []
C4 = []
C5 = []

for filename in os.listdir(directoryA):
    C1.append(filename)
    
for filename in os.listdir(directoryB):
    C2.append(filename)
    
for filename in os.listdir(directoryC):
    C3.append(filename)
    
for filename in os.listdir(directoryD):
    C4.append(filename)
    
for filename in os.listdir(directoryE):
    C5.append(filename)

def remove_prepositions(text):
    tokens = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(tokens)
    filtered = [word for word, pos in tagged if pos not in ['IN', 'TO']]
    return ' '.join(filtered)

stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()
stopwords = ["was","wasn't","was not","donot","don't","there","here","never","n't","s","been","will","would","wouldn't","hasn't","has not","haven't","have not","doesn't","had","hadn't","had not","upto","and","or","not","nt","that","this","it","i","why","who","how","what","when","he","she","their","you","but","because","then","which","him","her","their","where","could","an"]
translator = str.maketrans('', '', string.punctuation.replace('-', ''))

with open("C:\\Users\\DELL\\Desktop\\Ass3\\Stopword-List.txt", "r") as f:
    contents = f.read()
    words = contents.split()

    # Store the words in an array
    for word in words:
        stopwords.append(word)

with open(directory1, 'r') as f:
    remove = f.read()
    
# iterate over all files in the directory
for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    
    # read the contents of the file
    with open(filepath, 'r') as f:
        contents = f.read()
        if filename == "64831":
            contents = re.sub(r'^X', '', contents, flags=re.MULTILINE)
        name.append(filename)
        contents = re.sub(r'[^\w\s]', '', contents)
        contents = re.sub(r"\b\w*[0-9@]+\w*\b", "", contents)
        contents = nltk.word_tokenize(contents)
        contents = [stemmer.stem(word) for word in contents]
        contents = ' '.join(contents)
        #contents = " ".join([lemmatizer.lemmatize(word, pos='v') if pos.startswith('V') else lemmatizer.lemmatize(word) for word, pos in nltk.pos_tag(nltk.word_tokenize(contents))])
        contents = contents.translate(translator)
        contents = contents.lower()
        contents = remove_prepositions(contents)
        contents = ' '.join(word for word in contents.split() if word not in stopwords)
        
    # add the contents to the array
    file_contents.append(contents)

# Step 1: Split the preprocessed text data into individual words
sentences = [doc.split() for doc in file_contents]

# Step 2: Train a Word2Vec model on the list of sentences
model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=3)


# Step 3: Get the average word embedding for each document
document_embeddings = []
for doc in sentences:
    doc_embedding = np.mean([model.wv.get_vector(word) for word in doc if word in model.wv.key_to_index], axis=0)
    document_embeddings.append(doc_embedding)

# Step 4: Use K-means clustering to cluster the documents based on their average word embeddings
kmeans = KMeans(n_clusters=5, random_state=42).fit(document_embeddings)

predicted_cluster_labels = kmeans.labels_
predicted_cluster_labels+=1
true_labels = [int(name.split("_")[0]) - 1 for name in name]

cluster_true_pairs = list(zip(predicted_cluster_labels, true_labels))
cluster_counts = Counter(predicted_cluster_labels)

Ground_label = []

for x in range(0, len(name)):
    if name[x] in C1:
        Ground_label.append(1)
    elif name[x] in C2:
        Ground_label.append(2)
    elif name[x] in C3:
        Ground_label.append(3)
    elif name[x] in C4:
        Ground_label.append(4)
    else:
        Ground_label.append(5)
contingency = contingency_matrix(Ground_label, predicted_cluster_labels)
purity = np.sum(np.amax(contingency, axis=0)) / np.sum(contingency)
print('Purity via Word2Vec Baseline 3: ',purity*100, "%")


Purity via Word2Vec Baseline 3:  38.0 %


C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
